In [10]:
import cntk
import numpy
import cv2
from matplotlib import pyplot as plt
from PIL import Image

def create_reader(path, is_training, input_dim, num_label_classes):
    featureStream = cntk.io.StreamDef(field='features', shape=input_dim, is_sparse=False)
    deserailizer = cntk.io.CTFDeserializer(path, cntk.io.StreamDefs(features=featureStream))
    return cntk.io.MinibatchSource(deserailizer, randomize=is_training, max_sweeps=1, randomization_window_in_chunks=0)

def load_model(path):
    model = cntk.ops.functions.load_model(path)
    return model

def predict(model, data):
    return model.eval(data)

def do_predict(model_path, data_path):
    model = load_model(model_path)
    reader = create_reader(data_path, False, 3, 3)
    input_map = {
        input: reader.streams.features
    }
    data = reader.next_minibatch(63684, input_map=input_map)
    
    feature_data = data[input].asarray()
    #print("feature data",feature_data)
    w, h = 261, 244
    data = numpy.zeros((h, w, 3), dtype=numpy.uint8)
    for index, item in enumerate(feature_data):
        #print(index, item)
        i = index // 261
        j = index % 261
        data[i, j] = [item[0][0], item[0][1], item[0][2]]

    img = Image.fromarray(data, 'RGB')
    img.save('original.png')
    
    return predict(model, feature_data)

model_path = "./saved_model_exp_1/set-8.model"
data_path = "./test_rgb_file.txt"
predict_value = do_predict(model_path, data_path)

predict_value = predict_value.round()
#print("predict data ", predict_value)
print(len(predict_value))

img = Image.new('RGB', (261,244), "black") 
pixels = img.load() 

print(img.size[0])
print(img.size[1])

w, h = 261, 244
data = numpy.zeros((h, w, 3), dtype=numpy.uint8)
for index, item in enumerate(predict_value):
    i = index // 261
    j = index % 261
    data[i, j] = [item[0], item[1], item[2]]
    with open("predict_value.txt", 'a') as outfile:
                    outfile.write("{})R: {}, G: {}, B: {} \n".format(index+1, item[0], item[1], item[2]))

img = Image.fromarray(data, 'RGB')
img.save('my.png')

'''
for i in range(img.size[0]):    
    for j in range(img.size[1]):
        index = (j * 261) + i
        print(i, j, index)
        #pixels[i,j] = (predict_value[index][0], predict_value[index][1], predict_value[index][2]) 
        #print(pixels[i,j])

img.save("resultttttttttttt.png", "PNG")
'''
print("Finish")

1
261
244
Finish
